<a href="https://colab.research.google.com/github/GustavoMillerOS/Eda_Tech_Store/blob/main/tratamento_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importações e Carregamento dos Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

df_clientes = pd.read_csv("https://raw.githubusercontent.com/GustavoMillerOS/tratamento_dados/refs/heads/main/dados_clientes.csv")

#Observação dos Dados

In [ ]:
df_clientes.head()

In [ ]:
df_clientes.tail()

In [ ]:
df_clientes.info()

In [ ]:
df_clientes.describe()

In [ ]:
df_clientes.shape

In [ ]:
df_clientes.hist(bins = 100, figsize=(15,5))
plt.show()

#Tratamento

* Converter colunas de data para o formato correto e Criando colunas derivadas. ✅
* Verificar se há registros duplicados e Remover duplicatas mantendo os dados mais relevantes. ✅
* Utilizar boxplots e quartis para identificar outliers em colunas numericas. ✅
* Identificar colunas com valores nulos e Substituir, remover ou preencher valores ausentes de forma adequada. ✅
* Normalizar colunas numéricas e Converter colunas categóricas em variáveis numéricas com encoding. ✅
* Gerar Visualizações. ✅
* Exportação de Arquivos. ✅

###Converter colunas de data para o formato correto e Criando colunas derivadas.

In [ ]:
#Converter coluna Data para tipo datetime
df_clientes['data_compra'] = pd.to_datetime(df_clientes['data_compra'], dayfirst=True, format='mixed', errors='coerce')

In [ ]:
#Criando colunas derivadas ano, mês, dia da semana

df_clientes['ano_compra'] = df_clientes['data_compra'].dt.year
df_clientes['mes_compra'] = df_clientes['data_compra'].dt.month
df_clientes['dia_semana_compra'] = df_clientes['data_compra'].dt.day_name()
df_clientes.head()

###Verificar se há registros duplicados e Remover duplicatas mantendo os dados mais relevantes.

In [ ]:
#Verificando registros duplicados na tabela clientes
duplicados_cliente = df_clientes.duplicated().sum()

print(f"Registros duplicados na tabela clientes: {duplicados_cliente}")

In [ ]:
#Verificando registros duplicados utilizando PK id_clientes
duplicados_id_cliente = df_clientes.duplicated(subset=['id_cliente']).sum()

print(f"Registros duplicados pelo ID Cliente: {duplicados_id_cliente}")

In [ ]:
#Criando nova Coluna ID_cliente
df_clientes = df_clientes.reset_index(drop=True)
df_clientes['id_cliente_correto'] = df_clientes.index + 1000
df_clientes.head()

###Utilizar boxplots e quartis para identificar outliers em colunas numericas.

In [ ]:
#Analisando possiveis outliers em 'valor_compra'
Q1_compra = df_clientes['valor_compra'].quantile(0.25)
Q3_compra = df_clientes['valor_compra'].quantile(0.75)
IQR_compra = Q3_compra - Q1_compra

limite_inferior_compra = Q1_compra - 1.5 * IQR_compra
limite_superior_compra = Q3_compra + 1.5 * IQR_compra

print(f"\nValor Compra - Q1: {Q1_compra:.2f}, Q3: {Q3_compra:.2f}, IQR: {IQR_compra:.2f}")
print(f"Valor Compra - Limite Inferior Outlier: {limite_inferior_compra:.2f}, Limite Superior Outlier: {limite_superior_compra:.2f}")

outliers_compra = df_clientes[(df_clientes['valor_compra'] < limite_inferior_compra) | (df_clientes['valor_compra'] > limite_superior_compra)]
print(f"\nOutliers identificados no 'valor_compra': {len(outliers_compra)}")
if not outliers_compra.empty:
    display(outliers_compra[['id_cliente_correto', 'nome', 'valor_compra']])

In [ ]:
sns.boxplot(x=df_clientes["valor_compra"])
plt.suptitle("Distribuição Valor Compra")
plt.show()

In [ ]:
#Analisando possiveis outlier em 'renda_anual'
Q1_renda = df_clientes['renda_anual'].quantile(0.25)
Q3_renda = df_clientes['renda_anual'].quantile(0.75)
IQR_renda = Q3_renda - Q1_renda

limite_inferior_renda = Q1_renda - 1.5 * IQR_renda
limite_superior_renda = Q3_renda + 1.5 * IQR_renda

print(f"Renda Anual - Q1: {Q1_renda:.2f}, Q3: {Q3_renda:.2f}, IQR: {IQR_renda:.2f}")
print(f"Renda Anual - Limite Inferior Outlier: {limite_inferior_renda:.2f}, Limite Superior Outlier: {limite_superior_renda:.2f}")

outliers_renda = df_clientes[(df_clientes['renda_anual'] < limite_inferior_renda) | (df_clientes['renda_anual'] > limite_superior_renda)]
print(f"\nOutliers identificados na 'renda_anual': {len(outliers_renda)}")
if not outliers_renda.empty:
    display(outliers_renda[['id_cliente_correto', 'nome', 'renda_anual']])

In [ ]:
sns.boxplot(x=df_clientes["renda_anual"])
plt.suptitle("Distribuição Renda Anual")
plt.show()

#### **Observação:** Os outliers de valor de compra e de renda representam 6,67% de ambas as bases de dados. Por se tratar de um estudo de caso, decidiu-se manter esses valores




###Identificar colunas com valores nulos e Substituir, remover ou preencher valores ausentes de forma adequada.

In [ ]:
# Verificando valores Nulos.
df_clientes.isnull().sum()

In [ ]:
# Verificando Média e Mediana para tomada de decisão
metricas = df_clientes[['renda_anual', 'valor_compra', 'idade']].agg(['mean', 'median']).T
metricas.columns = ['Média', 'Mediana']
metricas.index = ['Renda Anual', 'Valor de Compra', 'Idade']

display(metricas.round(2))

In [ ]:
#Prenchendo idade com a Mediana
df_clientes['idade'].fillna(mediana_idade, inplace=True)

In [ ]:
#Prenchendo Renda Anual com a Mediana
df_clientes['renda_anual'] = df_clientes['renda_anual'].fillna(mediana_renda_anual)

In [ ]:
#Prenchendo Valor da Compra com a Mediana
df_clientes['valor_compra'] = df_clientes['valor_compra'].fillna(mediana_valor_compra)

In [ ]:
#Removendo datas ausentes
df_clientes.dropna(subset=['data_compra'], inplace=True)

In [ ]:
#Verificando se existem valores Nulos após ajustes
df_clientes.isnull().sum()

###Normalizar colunas numéricas e Converter colunas categóricas em variáveis numéricas com encoding.

In [ ]:
#Encoding
colunas_numericas = ['idade', 'renda_anual', 'valor_compra']
colunas_categoricas = ['genero', 'regiao', 'categoria_produto']

df_final = pd.get_dummies(df_clientes, columns=colunas_categoricas, drop_first=True)

In [ ]:
#Normalização
scaler = StandardScaler()
df_final[colunas_numericas] = scaler.fit_transform(df_final[colunas_numericas])

In [ ]:
# Visualização do resultado
display(df_final.head())

In [ ]:
#Análise de Correlação (Insight Estratégico)
plt.figure(figsize=(10, 6))

sns.heatmap(df_final.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')

plt.title('Correlação entre Variáveis (Apenas Numéricas)')
plt.show()

###Gerar Visualizações

In [ ]:
#Histograma mostrando a distribuição da renda dos clientes.
df_clientes['renda_anual'].hist(bins=30, color='skyblue', edgecolor='black', figsize=(10, 4))
plt.suptitle("Distribuição da Renda dos Clientes")
plt.show()

In [ ]:
#Boxplot comparando o gasto médio por categoria de produto.
plt.figure(figsize=(10, 4))
sns.boxplot(x='categoria_produto', y='valor_compra', data=df_clientes)
plt.title("Gasto Médio por Categoria de Produto")
plt.xlabel('Categoria do Produto')
plt.ylabel('Valor da Compra ($)')
plt.show()

In [ ]:
#Scatterplot mostrando a relação entre idade e valor gasto.
plt.figure(figsize=(10, 4))
sns.scatterplot(x='idade', y='valor_compra', data=df_clientes, hue='categoria_produto', s=100)
plt.title('Relação entre Idade e Valor Gasto')
plt.xlabel('Idade (Anos)')
plt.ylabel('Valor da Compra ($)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
#Heatmap de correlação entre variáveis numéricas.
corr = df_clientes[['idade', 'renda_anual', 'valor_compra']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Heatmap de Correlação entre Variáveis Numéricas')
plt.show()

###Exportação de Arquivos.

In [ ]:
#Exportando o dataset final limpo e transformado
df_final.to_csv('clientes_preparados_final.csv', index=False)
print("✅ Arquivo 'clientes_preparados_final.csv' exportado com sucesso!")

In [ ]:
#Criando um relatório em Excel com abas comparativas
with pd.ExcelWriter('relatorio_limpeza_clientes.xlsx') as writer:
    #Salvando a base original (usando a variável que continha os 30 registros)
    df_clientes.to_excel(writer, sheet_name='Antes_do_Tratamento', index=False)

    #Salvando a base final tratada
    df_final.to_excel(writer, sheet_name='Depois_do_Tratamento', index=False)

print("✅ Relatório 'relatorio_limpeza_clientes.xlsx' criado com sucesso!")

In [ ]:
from google.colab import files

#Baixar o dataset
files.download('clientes_preparados_final.csv')

#Baixar o relatório Excel
files.download('relatorio_limpeza_clientes.xlsx')